<a href="https://colab.research.google.com/github/Francoz-Charlotte/Spatial_teaching_CFediting/blob/master/2_webmaps_and_distributions_CFediting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Getting started ### 

<font color='orangered'> ~ déjà vu ~</font> if you haven't already done so: You need your own copy of this notebook. Go to "File" and 'save a copy in github' (give access if needed.... put it into the repository you made for this course). Now you have your own copy of the notebook. Click 'open in colab' to get started working on the practical exercise.

# Interactive maps and looking more at distributions

<font color='orangered'> ~ déjà vu ~</font> Last week, we focused on making mostly static maps, that is maps where you mostly just expect your user to look at the product you've prepared. Around a research question about iron age sites distribution in Central Italy, we focussed the practical on the purpose of the output (i.e. the map) through:
  * data exploration => filtering and creating data subset
  * visualisation => zoom level (visual balance), visual variables (symbols/forms, dimension, color) & weight
  * organisation => difference between geospatial and attribute data (role of attribute weighting)


<font color='magenta'> ~ new ~</font> This week we'll continue exploring types of output and we'll look at making maps where interactivity is part of the design. This will allow to spend some more time on that all important topic: **spatial distributions**. 


Central to this is our abibility to interrogate, manipulate and remodel the geospatial database and answer spatially explicit questions. This practical lab will provide you ways to do so by:

  * transforming database (combining, reducing by selection, merging)
  * creating layers


*Types of Distribution: Data Sampling*


<img src="https://github.com/Francoz-Charlotte/Spatial_teaching_CFediting/blob/master/Distribution.png?raw=1" width="500"/> </div> 

*Examples of Applications*

<img src="https://github.com/Francoz-Charlotte/Spatial_teaching_CFediting/blob/master/Examples%20of%20applications.png?raw=1" width="500"/> </div> 

##Start by getting tools, always



In [0]:
#codecell_Webmaps&Distributions_ImportUrLibraries

#Like last time, get the tools we need at the start
import pandas as pd
import folium
import numpy as np

in #codecell__Webmaps&Distributions_ImportUrLibraries:

<font color='orangered'> ~ déjà vu ~</font> Last week, we worked with folium, branca and pandas libraries. 

<font color='magenta'> ~ new ~</font> This week, you will still be working with panda and folium and be replacing branca library for [numpy](https://numpy.org/). 

## And then by getting the data

This week we are working with the data from the Antikythera survey project.


It's citation is: Bevan, A. and Conolly, J. 2012. Intensive Survey Data from Antikythera, Greece. **Journal of Open Archaeology Data** 1(1), DOI: http://dx.doi.org/10.5334/4f3bcb3f7f21d.

This data has been made available at the Archaeological Data Service (ADS) archive at https://archaeologydataservice.ac.uk/archives/view/antikythera_ahrc_2012/ and completely open for re-use. You can [see](https://archaeologydataservice.ac.uk/archives/view/antikythera_ahrc_2012/stats.cfm) how many times their database has been viewed and downloaded. 


### Open Data

Last week we mentioned open source software. Open data operates under the same broad ethos, and follows many of the same principles. Sharing, reuse, and attribution are key. If you continue to reuse the Antikythera data, be sure to continue to link back to and cite the source.

* Perhaps the most relevant example in the UK is **OS (Ordnance survey) OpenData** which was made freely available for the first time in 2010. Last review ([2018](https://www.ordnancesurvey.co.uk/business-government/tools-support/open-data-support)) counted 1.9 million open data downloads, the equivalent of 150 people download OS OpenData every day!

* 2015 has seen Environmental Agency made **lidar (light detection and ranging)** data available to the public, for free, as open data. Within the first year of release 500,000 lidar downloads were made equating to nearly 13 million km2 of data! 


### Working with other people's data: the case of Antikythera 

Have a quick look around the dataset as it's described on the ADS site. You'll notice that they've split up their dataset in ways that made sense to them at the time. Specifically they've divided up the artefact data into three discrete elements: 'pottery', 'lithics' and 'other' into separate files (very much like we did last week by filtering and creating a subset). This is a pretty normal archaeological data thing to do. 

Here's the trick, we want to focus on both **ceramics and small finds** and to look at these datasets together. This means you'll have to grab both of them and  combine them. You are organising the attribute data in order to reuse it for something new. This is thinking of a <font color='magenta'> **relational database model**</font>.

In [0]:
#codecell_Webmaps&Distributions_OrganisingUrdata

#Like last time, get the data we'll need at the start. I've been nice again and converted their coordinates to latitude and longitude for you. 
#You'll learn to do this yourself later in the course.

# we label the first dataset 'pottery'
pottery = pd.read_csv('https://raw.githubusercontent.com/ropitz/spatialarchaeology/master/data/antikythera_survey_pottery.csv')

# we label the second dataset 'small finds'
small_finds = pd.read_csv('https://raw.githubusercontent.com/ropitz/spatialarchaeology/master/data/antikythera_survey_small_finds.csv')



In [0]:
#codecell_Webmaps&Distributions_CheckingZeData

#let's check the individual pottery file.  
pottery.head()

In [0]:
#codecell_Webmaps&Distributions_CheckingZeData

#let's check the individual pottery file to see 
small_finds.head()



---


###Learning a new language – decomposing the code
<font color='orangered'> ~ déjà vu ~</font> in #codecell__Webmaps&Distributions_CheckingZeData:

* **=**    allows you to provide a new name, often more convenient than the link pathname
* **pd.read_csv** allows you to open a CSV format file

* **.head() function** take a peek at the first part of the dataset 


---


Here have a look at these 2 datasets and how they are structured. Also, the coordinates have been transformed for you from cartesian co-ordinates (grid system - shown in column *'Xsugg' &	'Ysugg'*) back to latitude/longitude (geodetic system - shown in column *'DDlat' &	'DDlong'*). Below is a very crude diagram demonstrating this transformation. 

*Relationship between Geographic (top left image)& Cartesian coordinates (bottom left image)*

<img src="https://github.com/Francoz-Charlotte/Spatial_teaching_CFediting/blob/master/x&y_webmaps&distribution_2.png?raw=1" width="350"/> </div><img src="https://github.com/Francoz-Charlotte/Spatial_teaching_CFediting/blob/master/Geographic%20&%20Cartesian%20Coordinates.png?raw=1" width="300"/> </div>



Whilst the conversion from geodetic to cartesian is fairly straightforward, converting cartesian to geodetic is a complex problem (& if you are interested on how this is done mathematically have a look at [this](https://www.movable-type.co.uk/scripts/latlong-os-gridref.html)).


---



In [0]:
#codecell_Webmaps&Distributions_Concatenation

# then we combine the two datasets together to make a big dataset we call 'survey data'
survey_data = pd.concat([pottery,small_finds], sort=False, ignore_index=True)

##Learning a new language – decomposing the code

Codecell__Webmaps&Distributions_Concatenation can be decomposed like this:
  
![](https://github.com/Francoz-Charlotte/Spatial_teaching_CFediting/blob/master/Webmaps&Distributions_Concatenation_.jpg?raw=1)



---


* **Reminder**: an array is a group of elements/objects/items organised as a set where columns (of equal size) are multiplied by rows (of equal size). The advantage of using arrays is that  all elements can be accessed at any time randomly. Last week, you had done something similar to call all items from the range [i] ( <font color='crimson'>in #codecell_makeabasicmap_BringingUrData2theMap </font>). This week, you are linking together 2 arrays. Pandas library provides various ways to combine together Series or DataFrame such as merge, join and concatenate ([see user guide](https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html)). 


---




Let's make sure nothing went wrong...

In [0]:
#codecell_Webmaps&Distributions_CheckingZeData

#check things loaded in and combined OK
survey_data.head()

# Now ask a question

Like we said last week, we are using maps and spatial analysis to pose, explore and respond to spatial questions. 

My question is a bit like last week's question. I want to know about how many sites are in each period, so I can try and understand changing patterns over time. 

However, you may have noticed when you read in the data that it's structure is a bit different from last week's data. Instead of each site belonging to one period, it's assigned with varying probability to several different periods. 

This is a totally legit archaeological thing to do. Many sites have activity from multiple periods, and depending on the available evidence, you might be more or less confident about the presence or absence of activity in a specific period. 

### So what do we do now?

We might start simply by assigning each site primarily to its 'most likely period'. 

This takes a few steps....


### step 1 - prepare the data

...you have to re-think the dataset to answer our question: **'how many sites are in each period?'**. Data cleaning is the most consuming part of any analysis - and not only in archaeology but for all sciences analyses in general.  


In [0]:
#codecell_Webmaps&Distributions_Subselect

# first we create a subset of our data that only contains the columns with information about time
# this is in part because we want to do some operations where everything has to be a number, and some of the other fields contain text
# it's also just to make things simpler when we look at them

survey_data_time = survey_data[['MNLN', 'FNEB1',	'EB2',	'LPrePal', 'FPal', 'SPal', 'TPal', 'PPalPG', 'Geom', 'Arch', 'Class', 'Hell', 'ERom', 'MRom', 'LRom', 'EByz', 'MByz', 'EVen', 'MVen', 'LVen', 'Recent', 'Other']]
survey_data_time.head()

---
####Learning a new language – decomposing the code 

<font color='orangered'> ~ déjà vu ~</font>  in codecell_Webmaps&Distributions_Subselect

**[ ]**    allows you to subselect within your new *'survey_data'* dataframe.
 
---

In [0]:
#codecell_Webmaps&Distributions_ChangingDataType

# if you were to look through this data
survey_data_time.dtypes

---
####Learning a new language – decomposing the code

In #codecell__Webmaps&Distributions_ChangingDataType, <font color='magenta'> ~ .dtypes ~</font> function allows you to see which type of data is in our *'survey_data_time'* dataframe.Three types of data are returned **int64** which are integers (whole numbers -numerical data), **float64** which are floating data (contains floating decimal points -numerical data) and **object** which is a string (not a number -categorical data). 

Looking further in this dataset, *'survey_data_time'*, we can also see that some that some fields contain missing values (NaN). Let's see what we can do about that and why it matters. 

---



In [0]:
#codecell_Webmaps&Distributions_AllNumbers

# null values can break number-based operations
# let's get rid of missing values and make sure everything is a number

survey_data_time.astype('float64').fillna(0)

---
####Learning a new language – decomposing the code

In the #codecell_Webmaps&Distributions_AllNumbers, you are making sure all these missing values, NaN, becomes null numbers (0=zero) . 

But why missing values need to be removed? Simply because you cannot apply maths to them (e.g. adding or multiplying columns together, reorder values of a column from max to min value, etc.).

And this is how it can be done:

<img src="https://github.com/Francoz-Charlotte/Spatial_teaching_CFediting/blob/master/Webmaps&Distributions_RemovingNaN_.jpg?raw=1" width="500"/>
</div>



---



### step 2 - Reshaping your data

Data transformation is an important thing to learn to do. 

Right now we have a bunch of columns with information about time. What we want is one single column that contains the most likely period - which is represented in each row by the column with the greatest value. 

*think about that for a moment*

Right now the 'most likely period' is represented by a number in each row, but that's not the piece of information we want in our new column - we want the name of the column that contains that number.Once you have the positions of the maximum values, you can extract them...

**In other words, we want to know the index positions of the elements (that satisfy the condition) and do whatever you want with it.** 


In [0]:
#codecell_Webmaps&Distributions_MaximumValue

#here we take the columns from all the different periods, get the one with the maximum value, 
def returncolname(row, colnames):
    return colnames[np.argmax(row.values)]
#and write that column's name to our new 'colmax' field
survey_data_time['colmax'] = survey_data_time.apply(lambda x: returncolname(x, survey_data_time.columns), axis=1)


---
####Learning a new language – decomposing the code

In the #codecell_Webmaps&Distributions_AllNumbers, you learn a new way to: 

#####1) define a function : 


<img src="https://github.com/Francoz-Charlotte/Spatial_teaching_CFediting/blob/master/Webmaps&Distributions_MaximumValue.png?raw=1" width="700"/> </div> 





#####2) pass/apply this function as an argument:

<img src="https://github.com/Francoz-Charlotte/Spatial_teaching_CFediting/blob/master/Webmaps&Distributions_MaximumValue_2.png?raw=1" width="900"/> </div>

---



#####*how does this really work?* 

Remember that we are working with arrays, so, getting maximum or minimum values can be schematised like this:

<img src="https://github.com/Francoz-Charlotte/Spatial_teaching_CFediting/blob/master/Arrays_.png?raw=1" width="500"/> </div>



---



In [0]:
#codecell_Webmaps&Distributions_CheckingZeData

#we can check it has all gone well
survey_data_time.head()

## Merging tables

OK now we have a single column, *'colmax'*, with the information we need - the most likely date. To create this column, we broke off some of our data (the columns with numbers) from the rest of the data (important descriptive text). We might well want to stick these two datasets back together before proceeding. 

**splitting and merging tables is another basic skill when working with data**




In [0]:
#codecell_Webmaps&Distributions_MergingZeData

#now we can also add our new column back to our original data table by doing a 'merge'
#create a new table 'survey_data_maxtime' by merging our original 'survey_data' with ONLY the 'colmax' column from our new table
survey_data_maxtime = pd.merge(survey_data, survey_data_time['colmax'], how='inner', left_index=True, right_index=True)


survey_data_maxtime.head()

####Learning a new language – decomposing the code

In the #codecell_Webmaps&Distributions_AllNumbers, you learned how to merge using pandas, and, if you want to further explore and apply pd.merge() to your data, check which [parameters](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html) apply.

<img src="https://github.com/Francoz-Charlotte/Spatial_teaching_CFediting/blob/master/Webmaps&Distributions_Merge.png?raw=1" width="900"/> </div>

## The curse of abbreviations

Have a look at the resulting table. What do all those column names mean? Right now you are probably justifiably confused. We'll be talking more about the mess that is 'other people's data' next week. For now, have a look at the documentation for these datasets at: https://archaeologydataservice.ac.uk/catalogue/adsdata/arch-1115-2/dissemination/csv/pottery/documentation/pottery.txt

You'll see they explain that many of those weird abbreviations are periods and that the number in each one represents the chance that a given find belongs to that period. Sometimes I wish people wouldn't use abbreviations like this, but they've defined them in their metadata file, so we can't compain too much.

## Finally, we make maps!


We're going to look at a couple different ways of making maps, because there are lots of tools we can use to do this.

### Maps for visualization and interpretation

Broadly speaking, there are two ways to approach interpreting spatial patterns. There's visualisation and interpretation, where you might visually compare distributions or densities or locations of two or more datasets by plotting them on a map and intepreting what you see. Then there's statistical analysis. 

We'll start with the tools to do the first one, and introduce statistical analysis later in the course. 

We'll also discuss the value of each approach, and when to apply it.

### As always, start with a question

As we said at the beginning of today, we're interested in change over time. Analysis Question: **How does the distribution of finds change between different periods?**




In [0]:
#codecell_Webmaps&Distributions_ImportUrLibraries

# we're going to get geopandas, another tool for making maps
!pip install geopandas


In [0]:
#codecell_Webmaps&Distributions_ImportUrLibraries

# get some more tools for making maps (and other things)

%matplotlib inline
import geopandas as gpd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

## Geopandas

We're going to introduce a new mapping tool - [geopandas](http://geopandas.org/gallery/index.html). It can do many of the same things as folium, which we were using in #MakingaBasicMap practical lab.

It's particularly useful for showing categorical data. 

We have seen above in #codecell_Webmaps&Distributions_ChangingDataType that the function **.dtype()** allows you to see which type of data is your columns. Data can be broadly split between numerical and categorical (see image below), and the difference that lies between them is crucial to grasp as it will define which tests and commands you can perform. 

<img src="http://2.bp.blogspot.com/-IUepWeQ3HDA/VO2LddsPkDI/AAAAAAAACNU/pyoF8xyaYsg/s1600/datatypes.png" width="500"/> </div>


We could start tryping to see and understand the distributions of our sites by periods simply by mapping the period labels with different colors.


In [0]:
#Codecell_Webmaps&Distributions_dataframesINTOgeodataframes

# take our big dataset from above and turn it 
# from a 'dataframe' which is the data that folium uses to make maps 
#into a 'geodataframe' which is the data geopandas uses to make maps

gdf_survey = gpd.GeoDataFrame(
    survey_data_maxtime, geometry=gpd.points_from_xy(survey_data_maxtime.DDLon, survey_data_maxtime.DDLat))
print(gdf_survey.head())

####Learning a new language – decomposing the code

In the #codecell_Webmaps&Distributions_dataframesINTOgeodataframes, <font color='magenta'> gpd.GeoDataFrame()~</font>  allows you to define the geometry (<font color='magenta'>geometry=</font>) of the new dataframe  *'survey_data_maxtime'*, so  it can be mapped. 

Here, the points geolocation is defined by the x and y centroids (<font color='magenta'>gpd.points_from_xy~</font> )  following their respective geodetic coordinates that can be found in column *'survey_data_maxtime.DDLon'* for the Xs and *'survey_data_maxtime.DDLat'* for Ys.

<font color='magenta'>print()~</font> allows you simply to see the results.

In [0]:
#Codecell_Webmaps&Distributions_PlotGeodataframes

#plot your data colouring the points by the period to which they belong (in column 'colmax')
#the plot requires you to define which type of data it is (categorical or numerical)
#figsize =(width, height)

gdf_survey.plot(column='colmax', categorical=True, legend=True, figsize=(15,15))

### Assess the resulting map

Is it useful? Why or why not?

Can you see the distribution of sites from individual periods easily?

Can you easily discern change over time?

I'm not overly convinced by the result here.

What other approach might we take?


Let's try something else


In [0]:
#codecell__Webmaps&Distributions_SplittingUrData

#Maybe it would be better to only look at two or three periods at a time

#let's select a subset of our periods to see change from the early bronze age to hellenistic to late roman

types = ['EB2','Hell','LRom']
classic = gdf_survey.loc[gdf_survey['colmax'].isin(types)]
classic.head()

in #codecell__Webmaps&Distributions_SplittingUrData, 


<font color='orangered'> ~ déjà vu ~</font>  the function .loc (<font color='magenta'>gdf_survey.loc~</font> ) is similar to .loc[] & .iloc[] used in pandas last week (in #codecell_makeabasicmap_BringingUrData2theMap). Note that loc using a label to index data whereas iloc uses an integer position. 

<font color='magenta'>.isin()~</font> command allows you to exclude data from 

In [0]:
#plot your data colouring the points by the period to which they belong
classic.plot(column='colmax', categorical=True, legend=True, figsize=(15,15))

## Thinking about data visualization and map design

That's a bit better perhaps. The map is less crowded.

Recall our discussions about how to design a map well. Clearly too much data introduces design problems.

Well, now we can see the distributions a bit, and maybe say something about change over time, but there are still a lot of dots, and it's pretty clear dots from some periods are hidden under dots from other periods and we have no way to separate them. 

**what we need here is layers, so we can group our data and work with it interactively**
*It would also be nice to have some context for all those dots*


In [0]:
#Like last time, we'll use folium and one of it's plugins
from folium.plugins import HeatMapWithTime


### Map Visualizations with Folium


To see the survey data in context and build our interactive maps, we'll start by generating the base map that will be used throughout this notebook.

'Basemaps' are generic background maps, like a satellite image or an image of the street map. You know the different backgrounds you can show on google maps? Those are 'basemaps'. 

Have a look around the web, and you'll see that most modern online maps  use a basemap, so we're going to do so as well.


In [0]:
#get the survey area centre, like you did last week, so you can centre the map where the data is located

location_survey=survey_data_maxtime['DDLat'].mean(), survey_data_maxtime['DDLon'].mean()
print(location_survey)


In [0]:
#define a basemap we can reuse. Use the coordiantes for the centre you generated just above to centre the basemap
#This is a variant on how we did things last time...

def generateBaseMap(default_location=[35.870086207930626, 23.301798820980512], default_zoom_start=11):
    base_map = folium.Map(location=default_location, control_scale=True, zoom_start=default_zoom_start)
    return base_map

### Review - basic map controls

Arguments:<br><br>
location: Define the default location to zoom at when rendering the map<br>
zoom_start: The zoom level that the map will default to when rendering the map<br>
control_scale: Shows the map scale for a given zoom level

In [69]:
#check the basemap is working
base_map = generateBaseMap()
base_map

In [0]:
#lets get the heatmap tool, like last time, let's also get a measure control so we can measure distance
from folium import plugins
from folium.plugins import HeatMap
from folium.plugins import MeasureControl

Let's start by comparing MRom to LRom, that is middle roman to late roman sites by putting their data in separate layers.

In [0]:
# make a layer for when each period is more than 50% likely, so you have all the sites that are probably in that period
survey_data_MRom = survey_data_maxtime.loc[(survey_data_maxtime['MRom'] > 50)]
survey_data_ERom = survey_data_maxtime.loc[(survey_data_maxtime['ERom'] > 50)]


### The concept of layers

We've introduced a new concept here. Maps have 'layers'. Each layer contains information and can be turned on and off. Think of this like a stack of transparent paper. Each sheet of paper is a layer, and can be added to or taken away from the stack. Their order can also be changed.

In [72]:
# like last time, make heatmaps, but one for each period,  put them in different layers.
base_map = generateBaseMap()
mrom = HeatMap(data=survey_data_MRom[['DDLat', 'DDLon', 'MRom']].groupby(['DDLat', 'DDLon']).sum().reset_index().values.tolist(), radius=8, max_zoom=13).add_to(base_map)
erom = HeatMap(data=survey_data_ERom[['DDLat', 'DDLon', 'ERom']].groupby(['DDLat', 'DDLon']).sum().reset_index().values.tolist(), radius=8, max_zoom=13).add_to(base_map)

#give the layers sensible names
mrom.layer_name = 'Middle Roman Distribution'
erom.layer_name = 'Early Roman Distribution'

# add the layer control. This is the tool that lets you turn different layers in your map on and off
folium.LayerControl().add_to(base_map)


In [73]:
#Now generate your map by calling it by its name
base_map

Now try and add some more layers to the map to show other periods!

In [0]:
# make a layer for when the max period is LRom or MRom to compare these periods
survey_data_lrommax = survey_data_maxtime.loc[(survey_data_maxtime['colmax'] =='LRom')]
survey_data_mrommax = survey_data_maxtime.loc[(survey_data_maxtime['colmax'] =='MRom')]


In [75]:

# like last time, make heatmaps, but one for each period,  put them in different layers
base_map = generateBaseMap()

lrommax = HeatMap(data=survey_data_lrommax[['DDLat', 'DDLon']].groupby(['DDLat', 'DDLon']).sum().reset_index().values.tolist(), radius=8, max_zoom=13).add_to(base_map)
mrommax = HeatMap(data=survey_data_mrommax[['DDLat', 'DDLon']].groupby(['DDLat', 'DDLon']).sum().reset_index().values.tolist(), radius=8, max_zoom=13).add_to(base_map)

#give the layers sensible names
lrommax.layer_name = 'Late Roman Distribution'
mrommax.layer_name = 'Middle Roman Distribution'

# add the layer control
folium.LayerControl().add_to(base_map)
base_map


# Adds a measure tool to the top right

base_map.add_child(MeasureControl())


# Making visual comparisons

Perhaps it will be more useful to be able to view our distributions and explore them side by side, to help us compare what is happening in these two periods.

In [0]:
# get another plugin for side by side maps
from folium.plugins import DualMap

In [77]:
# declare you are making a new map "m"

# set the location to the location of your survey, set your starting zoom
m = plugins.DualMap(location=location_survey, tiles=None, zoom_start=13)

# the dual maps plugin automatically defines two buddy maps, "m1" and "m2" which pan and zoom together

# give yourself some options in life for your base layers, add them to both maps 'm1' and 'm2' by just using "m"
folium.TileLayer('cartodbpositron').add_to(m)
folium.TileLayer('openstreetmap').add_to(m)


# like last time, make heatmaps, one for each period,  put them in different layers
# put one layer in the left hand map 'm' and the other in the right hand map 'm2'

lrommax = HeatMap(data=survey_data_lrommax[['DDLat', 'DDLon']].groupby(['DDLat', 'DDLon']).sum().reset_index().values.tolist(), radius=8, max_zoom=13).add_to(m.m1)
mrommax = HeatMap(data=survey_data_mrommax[['DDLat', 'DDLon']].groupby(['DDLat', 'DDLon']).sum().reset_index().values.tolist(), radius=8, max_zoom=13).add_to(m.m2)

#give the layers sensible names
lrommax.layer_name = 'Late Roman Distribution'
mrommax.layer_name = 'Middle Roman Distribution'

# layer control time
folium.LayerControl(collapsed=False).add_to(m)

# Adds a measure tool to the top right

m.add_child(MeasureControl())

#draw your side by side maps

m

## visualization and interpretation

Thought exercise: The results of these two maps should be similar but slightly different. What is making the difference?

How good are you at interpreting these distributions and comparing them visually?

*This is me hinting at you that you are going to end up wanting to use statistics eventually*


 

# Think about basic principles

The principles of what we've done this week are the same as the principles of what we did last week. 

I think it's important to learn to do things more than one way, and to adapt to slightly different tools. The software and code packages used for modern spatial analysis and mapping are pretty diverse and are always developing as people improve things. It doesn't make much sense to just learn one way of making maps mechanistically. The important thing is to understand the principles of what you're doing. ###which are?####

In any code package that is meant to be used for making maps, odds are good you will find a way to set the zoom level, set the centre starting location, and set the initial scale. 

You will be able to set up colour schemes, map attributes, and make layers. Knowing keywords and princples is the important thing. 

## The End

That's all for today. Be sure to save your copy of the notebook in your own repo so I can see it!

#LexiCode
In the past two practical labs, you have learned and experimented with  programming commands - !Remember to load first your libraries (folium, branca, pandas, geopandas, seaborn, matplotlib.pyplot and numpy) to use this new language! - that you can now reuse with your own datasets :


>Lexicode_MakingaBasicMap | Lexicode_Webmaps&Distributions
>--- | ---
>	==   () [] | pd.concat()
>.head_csv() | .dtype()
>.read_csv() | astype()
>mean()  | fillna()
>folium.Map | def return
>range() | .apply(lambda x:*function*,axis=)
>len() | pd.merge()
>iloc[]| how= , left_index= ,left_index= 
>.value_counts()| gpd.GeoDataFrame()
>if =:| geometry=gpd.points_from_xy
>elif =: |print() 
>else =:| .isin(types)
>folium.Marker()| 
>folium.Icon()| 
>folium.Circle| 
>popup= | 
>radius= | 
>.values.tolist() | 
>plugins.HeatMap()| 
>  | 


